# Create the inference wrapper
In order to use the trained model, we need to write a wrapper script that loads the model and feeds it the data coming from the configured data source.  
An example is already implemented under the `src` folder and in the `entrypoint.py` file.  
This notebook is purely for demonstrating how to write such a wrapper. Execution of this notebook is not expected, and changes being made in this tutorial will not be reflected in the final package!

To execute this notebook, you need the model created in notebook [10-CreateClassificationModel](10-CreateClassificationModel.ipynb).

## Load the model
The wrapper script must load the model first.

In [ ]:
import joblib

# the model sits in the models folder, so we need to use the relative path for the notebook to find it
# this will be changed in entrypoint.py, because the model will be in the same folder as the script
with open("../models/model.joblib", 'rb') as model_file:
    linear_reg = joblib.load(model_file)


## Create example payloads

In this section, we are preparing example payloads that will be used to test our inference model.  
The payload is provided by the `AI Inference Server`, and contains the input data from PROFINET IO as a list of dictionaries in this example.

First, we create `data_dicts` as is a list of dictionaries, where each dictionary represents a row in the original DataFrame.  
Each payload is a subset of the `data_dicts` list, containing an arbitrary chosen number of consecutive dictionaries (in this case, it's 60, so each payload contain exactly one minute of measurements).  
The `payloads` are created by taking slices of these dictionaries at a time.


In [ ]:
import pandas
import numpy as np

data = pandas.read_csv("../data/historical_data.csv")

input_tags = ["temperature_A", "temperature_B", "temperature_C", "valve_position_A", "valve_position_B"]
data = data[input_tags]

data_dicts = data.to_dict(orient='records')

payloads = [data_dicts[i:i + 60] for i in range(0, len(data_dicts), 60)]

## Processing input payloads

We define a `process_input(...)` function that acts as the entry point. The method uses our model to predict the pH value on each record found in a payload.

In [ ]:
def process_input(payload: list):
    
    for record in payload:  # record is a dictionary with the keys being the input_tags
        df = pandas.DataFrame(record, index=[0])
        phC = linear_reg.predict(df)
        print(phC)

process_input(payloads[0])  # processing the first payload which contains 1 minute of data

## Calculating valve control values

Based on the previous script, we expand the `process_input(...)` method so that the valve positions are also calculated based on the predictions of the model.

First, we define an expected pH value for container C as `phC_mean = 9.0`. Once the `process_input(...)` method is called with a payload, it calculates the average predicted pH value and compares it to the expected value by a ratio called the `flow_control_ratio`.  
Based on the value of this ratio, we adjust the angle positions of the valves in opposite directions.

In [ ]:
phC_mean = 9.0

def process_input(payload: list):
    
    phC_predictions = []
    for record in payload:  
        # calculating the predicted phC for each record in the payload
        df = pandas.DataFrame(record, index=[0])
        phC_predictions.append(linear_reg.predict(df))

    phC_predictions = np.array(phC_predictions).flatten()  # create a 1D numpy array
    flow_control_ratio = phC_predictions.mean() / phC_mean  # calculate the flow control ratio

    valve_position_A_values = np.array(df['valve_position_A']).flatten()
    valve_position_A = valve_position_A_values.mean() * flow_control_ratio  # adjust the valve position A

    valve_position_B_values = np.array(df['valve_position_B']).flatten()
    valve_position_B = valve_position_B_values.mean() / flow_control_ratio  # adjust the valve position B

    return {"valve_position_A": valve_position_A, "valve_position_B": valve_position_B, "predicted_phC": phC_predictions.mean()}

print(process_input(payloads[1])) # processing the second payload

The return object of the `process_input(..)` must be a dictionary, containing the variables we want to provide for `AI Inference Server` as an output.  
This way `AI Inference Server` will be able to forward the data to the mapped `Data Connectors`.

The Python script [entrypoint.py](../src/inference/entrypoint.py) serves as the entrypoint for the pipeline we are going to build in the [30-CreatePipelinePackage](30-CreatePipelinePackage.ipynb) notebook.  
The python script contains the `process_input(...)` method, as well as as the necessary imports and loading our trained model.